In [14]:
import json
from tqdm import tqdm

# 1. 데이터 로드
with open("data/output/review.json", "r", encoding="utf-8") as f:
    reviews = [json.loads(line) for line in f]

with open("data/output/user.json", "r", encoding="utf-8") as f:
    users = [json.loads(line) for line in f]

with open("data/output/business.json", "r", encoding="utf-8") as f:
    businesses = [json.loads(line) for line in f]


In [15]:
import json
import pandas as pd
from tqdm import tqdm


# 2. DataFrame으로 변환
df_reviews = pd.DataFrame(reviews)
df_users = pd.DataFrame(users)
df_businesses = pd.DataFrame(businesses)

# 3. 정보 출력 함수
def print_info(df, name):
    print(f"\n📌 {name} 정보")
    print("-" * 40)
    print(df.info())
    print(df.head(2))

# 4. 각 파일의 정보 출력
print_info(df_reviews, "리뷰 데이터")
print_info(df_users, "사용자 데이터")
print_info(df_businesses, "음식점 데이터")



📌 리뷰 데이터 정보
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732435 entries, 0 to 732434
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   review_id     732435 non-null  object
 1   user_id       732435 non-null  object
 2   business_id   732435 non-null  object
 3   stars         732435 non-null  int64 
 4   useful        732435 non-null  int64 
 5   text          732435 non-null  object
 6   date          732435 non-null  int64 
 7   token_length  732435 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 44.7+ MB
None
                review_id                 user_id             business_id  \
0  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
1  JrIxlS1TzJ-iCu79ul40cQ  eUta8W_HdHMXPzLBBZhL1A  04UD14gamNjLY0IDYVhHJg   

   stars  useful                                               text  \
0      5       1  Wow!  Yummy, different,  deli

In [16]:
# user_id 매칭률 확인
review_user_ids = set([r["user_id"] for r in reviews])
user_ids = set([u["user_id"] for u in users])
print("✅ user_id 매칭 개수:", len(review_user_ids & user_ids))

# business_id 매칭률 확인
review_biz_ids = set([r["business_id"] for r in reviews])
biz_ids = set([b["business_id"] for b in businesses])
print("✅ business_id 매칭 개수:", len(review_biz_ids & biz_ids))


✅ user_id 매칭 개수: 28518
✅ business_id 매칭 개수: 6855


In [17]:
import json
from tqdm import tqdm

# 1. JSONL 로드 함수
def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

# 2. 경로 설정
review_path = "data/output/review.json"
user_path = "data/output/user.json"
business_path = "data/output/business.json"
output_path = "data/output/merged_dataset.json"

# 3. 파일 로딩
reviews = load_jsonl(review_path)
users = load_jsonl(user_path)
businesses = load_jsonl(business_path)

# 4. Dict 인덱싱
user_dict = {u["user_id"]: u for u in users}
business_dict = {b["business_id"]: b for b in businesses}

valid_user_ids = set(user_dict.keys())
valid_business_ids = set(business_dict.keys())

# ✅ 5. 유효한 리뷰 필터링
valid_reviews = [
    r for r in reviews
    if r.get("user_id") in valid_user_ids and
       r.get("business_id") in valid_business_ids and
       r.get("review_id")
]

# 6. 병합
merged_data = []
for r in tqdm(valid_reviews, desc="병합 중"):
    uid = r["user_id"]
    bid = r["business_id"]
    u = user_dict[uid]
    b = business_dict[bid]

    merged_entry = r.copy()  # review 필드 복사

    # user_ prefix
    for k, v in u.items():
        merged_entry[f"user_{k}"] = v

    # business_ prefix
    for k, v in b.items():
        merged_entry[f"business_{k}"] = v

    merged_data.append(merged_entry)

# 7. 저장 (JSONL)
with open(output_path, "w", encoding="utf-8") as f:
    for row in merged_data:
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

# 8. 결과 확인
print(f"✅ 병합 완료: {output_path}")
print(f"총 병합 리뷰 수: {len(merged_data)}")
print(f"제외된 리뷰 수: {len(reviews) - len(valid_reviews)}")
print(f"최종 컬럼 수: {len(merged_data[0]) if merged_data else 0}")
print("예시 컬럼:", list(merged_data[0].keys())[:10] if merged_data else "없음")


병합 중: 100%|██████████| 451394/451394 [00:02<00:00, 217967.21it/s]


✅ 병합 완료: data/output/merged_dataset.json
총 병합 리뷰 수: 451394
제외된 리뷰 수: 281041
최종 컬럼 수: 22
예시 컬럼: ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'text', 'date', 'token_length', 'user_user_id', 'user_name']


In [18]:
import json
import pandas as pd

# 1. JSONL 파일 경로
path = "data/output/merged_dataset.json"  # 필요 시 경로 수정

# 2. JSONL 파일 읽기
with open(path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]
df = pd.DataFrame(data)

# 3. 중복/불필요한 컬럼 제거
drop_cols = ["user_user_id", "business_business_id"]
df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)

# 4. 컬럼 이름 리네이밍 (명확하게 하기)
df.rename(columns={
    "stars": "review_stars",
    "useful": "review_useful",
    "date": "review_date"
}, inplace=True)

# 5. JSONL로 저장
output_path = "data/output/merged_dataset.json"
with open(output_path, "w", encoding="utf-8") as f:
    for row in df.to_dict(orient="records"):
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

# 6. 요약 출력
print("✅ 정리 완료: JSONL 형식으로 저장")
print(f"저장 경로: {output_path}")
print("총 샘플 수:", len(df))
print("최종 컬럼 수:", len(df.columns))
print("샘플 컬럼:", df.columns[:10].tolist())

✅ 정리 완료: JSONL 형식으로 저장
저장 경로: data/output/merged_dataset.json
총 샘플 수: 451394
최종 컬럼 수: 20
샘플 컬럼: ['review_id', 'user_id', 'business_id', 'review_stars', 'review_useful', 'text', 'review_date', 'token_length', 'user_name', 'user_review_count']


In [19]:
import pandas as pd

# 2. 공백 또는 NaN 개수 계산 함수
def count_empty_or_blank(series):
    return series.apply(lambda x: pd.isna(x) or (isinstance(x, str) and x.strip() == "")).sum()

# 3. 각 컬럼별 결측/공백 수 확인
empty_counts = df.apply(count_empty_or_blank)

# 4. 출력
print("📊 각 컬럼별 공백 또는 결측치 개수:")
print(empty_counts)

📊 각 컬럼별 공백 또는 결측치 개수:
review_id                0
user_id                  0
business_id              0
review_stars             0
review_useful            0
text                     0
review_date              0
token_length             0
user_name                0
user_review_count        0
user_useful              0
user_average_stars       0
business_name            0
business_address         0
business_city            0
business_state           0
business_stars           0
business_review_count    0
business_is_open         0
business_categories      0
dtype: int64


In [20]:
import json
import pandas as pd

# 1. JSONL 파일 경로
path = "data/output/merged_dataset.json"  # 필요 시 경로 수정

# 2. JSONL 파일 읽기
with open(path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# 3. pandas DataFrame으로 변환
df = pd.DataFrame(data)

# 4. 기본 정보 출력
print("📊 [기본 정보]")
print(df.info())
print()

# 5. 컬럼 목록
print("🧾 [컬럼 목록]")
print(df.columns.tolist())
print()

# 6. 예시 데이터
print("🔍 [샘플 데이터]")
print(df.head(3).to_string(index=False))

📊 [기본 정보]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451394 entries, 0 to 451393
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   review_id              451394 non-null  object 
 1   user_id                451394 non-null  object 
 2   business_id            451394 non-null  object 
 3   review_stars           451394 non-null  int64  
 4   review_useful          451394 non-null  int64  
 5   text                   451394 non-null  object 
 6   review_date            451394 non-null  int64  
 7   token_length           451394 non-null  int64  
 8   user_name              451394 non-null  object 
 9   user_review_count      451394 non-null  int64  
 10  user_useful            451394 non-null  int64  
 11  user_average_stars     451394 non-null  float64
 12  business_name          451394 non-null  object 
 13  business_address       451394 non-null  object 
 14  business_city          451